# <center> R4.04 Méthodes d'optimisation <br> TP2 - SAT Solvers </center>
<center> 2022/2023 - Thibault Godin & Lucie Naert </center>
<center> IUT de Vannes, BUT Informatique </center>

In [1]:
import numpy as np 
import pycosat
import itertools
import sys, getopt 
import time
from pprint import pprint
import math

import copy

*********

# $\mathbf{ SAT}$


En première année, vous avez vu les concepts d'assertions et de formules (booléennes) logiques. Par exemple la formule

$$A :=  (\neg P \wedge Q) \vee  (P\wedge \neg Q) $$


qui a pour table de vérité <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) :


|P|Q|A|
| ------------- |:-------------:| -----:|
|V|V|F|
|V|F|V|
|F|V|V|
|F|F|F|


Étant donnée une assertion $A$, on dira que l'assertion est _satisfaisable_ (satisfiable) s'il existe des valeurs de vérités pour les variables atomiques (ici $P$ et $Q$) telle que $A$ soit vraie. Dans notre exemple $A$ est satisfaisable, et un _témoin_ (witness) est le choix de valeurs $P=V$ et $Q=F$. 


La question _"étant donnée une assertion logique $A$, déterminer si elle est satisfaisable"_ est un problème majeur en informatique, appelé [SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_SAT)


<a name="cite_note-1"></a>1. [^](#cite_ref-1) On appelle généralement cet opérateur xor (ou exclusif)



**_question 1_**
Déterminer si les assertions suivantes sont satisfaisables, et donner un témoin si elle le sont.


- $A_1 :=  \neg (P \wedge Q) \vee R $
- $A_2 :=  (P \Longleftrightarrow Q) \wedge (P \oplus Q) $
- $A_3 :=  \neg (P \wedge Q) \vee P $
- $A_4 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$


Réponse : 


*********

## CNF

Pour les problèmes de satisfiabilité, on a généralement un grand nombre de variables booléennes. On les notera généralement $x_1, x_2, \dots, x_n$, et on dira que $n$ est la _taille_ de l'instance.
De plus, on a vu en R1.06 que toute formule peut s'écrire à l'aide des seules opérations $\neg, \vee, \wedge$, et l'on peut même exiger que les $\neg$ ne porte que sur des propositions atomiques (les $x_i$).


**_question 2_**

Transformer les propositions suivantes de manière à n'utiliser que les opérations $\neg, \vee, \wedge$, et que les $\neg$ ne porte que sur des propositions atomiques.
- $A_1 :=  \neg (P \wedge Q) \vee P $
- $A_1 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$
- $A_3 := \neg (P \Longleftarrow Q)$
- $A_4 :=  (P \Longleftrightarrow Q) \vee \neg P $

Réponse:



Pour faciliter la saisie des assertion, on va normaliser un peu plus encore : on va demander à ce que nos formules soient des conjonctions (des "et") de disjonctions (des "ou") de variables. On appelle cette forme [CNF](https://fr.wikipedia.org/wiki/Forme_normale_conjonctive).<a name="cite_ref-3"></a>[<sup>[2]</sup>](#cite_note-3)

Par exemple $(x_1 \vee x_2 \vee \neg x_3) \wedge (x_1 \vee x_3)$ et  $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ est en CNF,
tandis que $ \neg(x_1 \wedge x_2 \wedge \neg x_3) \vee (x_1 \wedge x_3)$ et $(x_1 \wedge \neg x_5 \wedge x_4) \vee  (\neg x_1 \wedge  x_5 \wedge x_3 \wedge x_4) \vee  (\neg x_3 \wedge \neg x_4)$ (le dernier exemple est en forme normale disjonctive, DNF)


On appelera _clause_ les parties entre deux $\wedge$ consécutifs.

<a name="cite_note-3"></a> 2. [^](#cite_ref-3) On peut aller encore plus loin en imposant que la formule soit une conjonction (un "et") de disjonctions (des "ou") d'au plus $3$ variables. On optient le problème [3-SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) qui est équivalent au problème $\mathbf{SAT}$

*********
**_question 3_**

Mettre $A_4$ en CNF.

Réponse : 


Une manière d'encoder une formule booléenne est la suivante : chaque clause est encodée dans un tableau d'entiers, la variable $x_5$ étant représentée par <tt>5</tt> et la variable $\neg x_5$ étant représentée par <tt>-5</tt>


Par exemple :

In [2]:
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]

représente la formule 


 $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ 
 
 
Ici, on a 5 variables et 3 clauses, la première clause étant $(x_1 \vee \neg x_5 \vee x_4)$. 

*********


On définit une fonction pour afficher des CNF et une fonction retournant le nombre de variables et de clauses d'une formule en CNF

In [3]:
def display(cnf):
    monCnf = "("
    for i in range(len(cnf)):
        for j in range(len(cnf[i])):
            if cnf[i][j] < 0:
                monCnf += "non "
            monCnf += "x"+str(abs(cnf[i][j]))
            if j != len(cnf[i]) - 1:
                monCnf +=" ou "
        if i != len(cnf) -1 :
            monCnf += ") et ("
        else :
            monCnf += ")"
    return monCnf

strcnf = display(cnf)
print(strcnf)


def cnflen(cnf):
    #returns the number of variables and of clauses of a boolean formula given under CNF
    return max([abs(max(max(cnf, key=max))),abs(min(min(cnf, key=min)))]),len(cnf)

nbVar, nbClause = cnflen(cnf)

print("Nombre de variable : ",nbVar, "\nNombre de clauses : ", nbClause)



(x1 ou non x5 ou x4) et (non x1 ou x5 ou x3 ou x4) et (non x3 ou non x4)
Nombre de variable :  5 
Nombre de clauses :  3


*********
**_question 4_**


Essayer de résoudre quelques instances en CNF sur [SAT Game](http://www.cril.univ-artois.fr/~roussel/satgame/satgame.php?level=2&lang=fr)


*********

# $\mathbf{SAT}$-Solver



La résolution algorithmique d'un problème $\mathbf{SAT}$ est assez facile : on peut tester tous les cas possibles, jusqu'à ce que l'on trouve un témoin ou que l'on ait essayé toutes les combinaisons. S'il y a $n$ variables, il n'y a "que" $2^n$ cas à tester.


*********

**_question 5_**


1. Écrire une fonction `Eval(cnf,wit)` qui prend en entrée `cnf` une formule booléenne en CNF à $n$ variables et `wit` un tableau de valeurs booléennes et retourne l'évaluation de la formule sur ces valeurs. Regarder bien le tests faits en dessous pour comprendre les types des paramètres.
2. Faire 3 tests supplémentaires : 
    - un test avec un premier CNF et un ensemble de valeurs pour lesquelles l'évaluation est "True"
    - un test avec le même CNF et un ensemble de valeurs pour lesquelles l'évaluation est "False"
    - un test avec un autre CNF

In [4]:
def Eval(cnf,wit):
    res=True
    #Todo
    return bool(res)

#Doit retourner Faux
witTest= [1,0]
cnfTest = [[-1, 2],[2]]
print(display(cnfTest))
print(cnflen(cnfTest))
print(Eval(cnfTest,witTest))

#Todo

(non x1 ou x2) et (x2)
(2, 2)
True


*********

**_question 6_**


Écrire une fonction `bruteSAT(cnf, verbose = False)` qui essaie de resoudre une formule en CNF par force-brute (en essayant toutes les possibilités) et renvoie un témoin si la formule est satisfaisable et -1 sinon. Si `verbose = True` et qu'un témoin est trouvé, le nombre d'essai avant de trouver un témoin doit être affiché ainsi que le nombre maximum d'essais possibles.

_On pourra utiliser le module itertool.product de python_


In [5]:
def bruteSAT(cnf):
    return -1


print(display(cnf))
print(bruteSAT(cnf))
print(bruteSAT([[1,2]]))
print(bruteSAT([[1],[2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2],[-1,-2]]))



(x1 ou non x5 ou x4) et (non x1 ou x5 ou x3 ou x4) et (non x3 ou non x4)
-1
-1
-1
-1
-1


*********

## PycoSAT

On ne sait pas aujourd'hui s'il existe un algorithme polynômial (_ie_ s'executant en temps raisonnable) pour résoudre $\mathbf{SAT}$. Cependant ce problème, bien qu'abstrait et théorique, est central en informatique ;  et on peut résoudre beaucoup de problèmes (qui paraissent pourtant complétement différents) en les traduisant en des formules logiques.
On verra partie 2 l'exemple du Sudoku, mais bien d'autres [problèmes](https://fr.wikipedia.org/wiki/Liste_de_probl%C3%A8mes_NP-complets) (3-coloration, emplois du temps ...) pourraient être résolus <a name="cite_ref-2"></a>[<sup>[3]</sup>](#cite_note-2) grâce à $\mathbf{SAT}$ (plus précisément à $\mathbf{3-SAT}$, qui [sert de référence](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) pour la [$\mathbf{NP}$-complétude](https://fr.wikipedia.org/wiki/Probl%C3%A8me_NP-complet))





<a name="cite_note-2"></a>3. [^](#cite_ref-2) les problèmes polynomiaux peuvent aussi être résolu par $\mathbf{SAT}$, ce n'est cependant pas pertinent pusiqu'ils sont justement polynomiaux tandis que $\mathbf{SAT}$ peut être exponentiel (sauf si $\mathbf{P} = \mathbf{NP})$

*********


En raison de cette importance, de nombreuses personnes cherchent à optimiser la résolution de formule, en créant des [SAT-Solver](https://en.wikipedia.org/wiki/SAT_solver) les plus efficaces possibles.

Dans ce TP, nous allons utiliser **pycosat**
https://pypi.org/project/pycosat/

La syntaxe de pycosat est légérement différente de la notre, comme le montre l'exemple suivant :


In [6]:
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]
pycosat.solve(cnf)

[1, -2, -3, -4, 5]

`[1, -2, -3, -4, 5]` est une solution possible trouvée par pycosat. Elle équivaut à un $[1,0,0,0,1]$ avec notre notation.

### Solutions multiples

In [7]:
for sol in pycosat.itersolve(cnf):
    print(sol)

[1, -2, -3, -4, 5]
[1, -2, -3, 4, -5]
[1, -2, -3, 4, 5]
[1, -2, 3, -4, -5]
[1, -2, 3, -4, 5]
[1, 2, 3, -4, -5]
[1, 2, 3, -4, 5]
[1, 2, -3, -4, 5]
[1, 2, -3, 4, -5]
[1, 2, -3, 4, 5]
[-1, 2, -3, 4, -5]
[-1, 2, -3, 4, 5]
[-1, 2, -3, -4, -5]
[-1, 2, 3, -4, -5]
[-1, -2, 3, -4, -5]
[-1, -2, -3, -4, -5]
[-1, -2, -3, 4, 5]
[-1, -2, -3, 4, -5]


*********


# Sudoku


Le problème général (sur une grille $n\times n$) du sudoku est $\mathbf{NP}$-complet, on ne connait pas d'algorithme efficace pour le résoudre. Il est plus simple de vérifier qu'une solution fonctionne que d'en trouver une.
Une approche classique de résolution est le _backtracking_, nous allons de nôtre côté utiliser pycosat.

On se donne quelques grilles d'exemple (vous pouvez créer les vôtres).

In [8]:

#https://github.com/taufanardi/sudoku-sat-solver/blob/master/Sudoku.py


easy = [[0, 0, 0, 1, 0, 9, 4, 2, 7],
        [1, 0, 9, 8, 0, 0, 0, 0, 6],
        [0, 0, 7, 0, 5, 0, 1, 0, 8],
        [0, 5, 6, 0, 0, 0, 0, 8, 2],
        [0, 0, 0, 0, 2, 0, 0, 0, 0],
        [9, 4, 0, 0, 0, 0, 6, 1, 0],
        [7, 0, 4, 0, 6, 0, 9, 0, 0],
        [6, 0, 0, 0, 0, 8, 2, 0, 5],
        [2, 9, 5, 3, 0, 1, 0, 0, 0]]

medium = [[5, 8, 0, 0, 0, 1, 0, 0, 0],
        [0, 3, 0, 0, 6, 0, 0, 7, 0],
        [9, 0, 0, 3, 2, 0, 1, 0, 6],
        [0, 0, 0, 0, 0, 0, 0, 5, 0],
        [3, 0, 9, 0, 0, 0, 2, 0, 1],
        [0, 5, 0, 0, 0, 0, 0, 0, 0],
        [6, 0, 2, 0, 5, 7, 0, 0, 8],
        [0, 4, 0, 0, 8, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 6, 5]]

evil = [[0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]]

hard = [[0, 2, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 0, 6, 0, 1, 0, 0, 0],
        [0, 6, 8, 2, 0, 0, 0, 0, 5],
        [0, 0, 9, 0, 0, 8, 3, 0, 0],
        [0, 4, 6, 0, 0, 0, 7, 5, 0],
        [0, 0, 1, 3, 0, 0, 4, 0, 0],
        [9, 0, 0, 0, 0, 7, 5, 1, 0],
        [0, 0, 0, 1, 0, 4, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 9, 0]]

blank = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]
    




On va encoder une grille de la manière suivante : 

Dans ce problème, les variable $x_{(i,j,d)}$ signifie "La cellule (i,j) contient d". 

Ainsi, la variable $x_{(1,3,2)}$ signifie "La cellule à la ligne 1 et colonne 3 contient un 2"

et $\neg x_{(1,3,2)}$ signifie évidemment "La cellule à la ligne 1 et colonne 3 ne contient pas un 2"

Malheureusement c'est une représentation $3$D (une variable est désignée par 3 nombres i, j, d), et nos CNF sont $1$d (un nombre désigne une variable). La fonction `x(i,j,d)` permet donc de faire une transformation 3D vers 1D

$x_{(i,j,d)} \leadsto 81*(i-1) + 9*(j-1) + d$

(Transformation pour que chaque case ait un numéro unique)

Evidemment, vous utiliserez la notation $x(i,j,d)$, beaucoup plus lisible.




In [9]:
# variable "La cellule (i,j) contient d"
# exemple:  x(5,2,7) signifie que la cellule à la ligne 5 et colonne 2 contient un 7 
# Pour pycosat, ce sera la variable 340 -> 81*(5-1) + 9*(2-1) + 7
def x(i, j, d): 
    return 81 * (i - 1) + 9 * (j - 1) + d

print(x(5,2,7))


340


**_question 7_**

Ecrire une fonction `gridToCNF(grid)` qui transforme une grille de Sudoku pré-remplie en conjonction de clauses. 

In [10]:
def gridToCNF(grid=blank):
    #turns a grid into a CNF
    cnf = []
    #Todo
    return cnf


**_question 8_**


Ecrire en python les formules CNF correspondant aux assertions : 
1. "chaque cellule doit contenir un nombre" (remplir `auMoinsUnNb()`)
2. "une cellule ne peut pas avoir plus d'un nombre" (remplir `auPlusUnNb()`)

In [11]:
def auMoinsUnNb() :
    cnf= []
    #todo
    return cnf
        
try:
    assert len(auMoinsUnNb()) == 81
    print("En tout cas, le nombre de clauses est correct...")
except:
    print("Erreur")

Erreur


In [12]:
def auPlusUnNb() :
    cnf= []
    #todo
                
    return cnf

try:
    assert len(verifLigne()) == 2916
    print("En tout cas, le nombre de clauses parait correct...")
except:
    print("Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi")
    



Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi


**_question 9_**

Quelles sont les règles du Sudoku ? Proposer et implémenter les 3 règles du Sudoku sous forme de CNF

In [13]:
def verifLigne() :
    cnf= []
    #todo         
    return cnf

try:
    assert len(verifLigne()) == 2916
    print("En tout cas, le nombre de clauses parait correct...")
except:
    print("Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi")
    




Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi


In [14]:
def verifColonne() :
    cnf= []
    #todo
    return cnf

try:
    assert len(verifColonne()) == 2916
    print("En tout cas, le nombre de clauses parait correct...")
except:
    print("Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi")
    



Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi


In [15]:
def verifCarre() :
    cnf= []
    #todo
                
    return cnf

try:
    assert len(verifCarre()) == 2916
    print("En tout cas, le nombre de clauses parait correct...")
except:
    print("Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi")
  

Erreur, optimisation possible ou vous avez une autre (potentiellement meilleure) solution que moi


Les fonctions suivantes assemblent les fonctions développées précédemment pour calculer et afficher la solution d'un Sudoku quand il est satisfaisable.

In [16]:
def sudokuClauses(grid) : 
    return gridToCNF(grid)+ auMoinsUnNb() + auPlusUnNb() + verifLigne() + verifColonne() + verifCarre()

def solve(grid=blank):

    # solve the SAT problem
    clauses = sudokuClauses(grid)
    #print(clauses)
    
    start = time.time()
    sol = set(pycosat.solve(clauses))
    end = time.time()
    print("Time: "+str(end - start))
    

    gridRes = copy.deepcopy(grid)
    for i in range(1, 10):
        for j in range(1, 10):
            gridRes[i - 1][j - 1] = read_cell(sol,i, j)
    return gridRes

            
# A partir d'une grille-solution (en code pycosat), cette fonction permet de retourner la valeur d'une cellule dont on donne les coordonées
def read_cell(sol,i, j):
    # return the digit of cell i, j according to the solution
    for d in range(1, 10):
        if x(i, j, d) in sol:
            return d
            
def solve_problem(problemset):
    print('Problem:') 
    pprint(problemset)  
    new_grid = solve(problemset) 
    print('Answer:')
    pprint(new_grid) 

    
    


**_question 10_**

Vous pouvez maintenant tester la modélisation de votre problème.

In [17]:
solve_problem(hard)

Problem:
[[0, 2, 0, 0, 0, 0, 0, 3, 0],
 [0, 0, 0, 6, 0, 1, 0, 0, 0],
 [0, 6, 8, 2, 0, 0, 0, 0, 5],
 [0, 0, 9, 0, 0, 8, 3, 0, 0],
 [0, 4, 6, 0, 0, 0, 7, 5, 0],
 [0, 0, 1, 3, 0, 0, 4, 0, 0],
 [9, 0, 0, 0, 0, 7, 5, 1, 0],
 [0, 0, 0, 1, 0, 4, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 9, 0]]
Time: 0.0
Answer:
[[None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None]]


### Extensions suggérées :


#### Génération aléatoire de grilles (résolubles)

à l'aide des algorithmes précédents, écrire une fonction engendrant des grilles partielles de sudoku (résolubles)



#### 3-Coloriage

De nombreux problèmes importants en informatique (les problèmes inclus dans $\mathbf{NP}$ sont _réductibles_ à 3-SAT, c'est-à-dire qu'on peut les convertir en un problème 3-SAT et vis-et-versa (en temps polynomial, avec un nombre de d'inconnues polynomial). 

On peut donc (essayer de) résoudre de nombreux problème avec 3-SAT. Parmi eux la 3-coloration (étant donné un graphe $G$, admet-il une 3 coloration ?)

Écrire un programme qui, étant donné un graphe $G$ transforme 3-col en 3-SAT, résout le prolème à l'aide d'un SAT-solver, puis donne la solution en terme de graphe).


_note :_ Pour le sens inverse, vous pouvez vous référer à https://cgi.csc.liv.ac.uk/~igor/COMP309/3CP.pdf par exemple